In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import pandas as pd


In [ ]:

directory = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/text/81_90'

In [4]:
def replace_text(file_path, company_name, year):
    with open(file_path, 'r') as file:
        content = file.readlines()

    updated_content = []
    for line in content:
        
        updated_line = re.sub(re.escape(company_name), 'Company', line, flags=re.IGNORECASE)

        
        if re.search(r'\bprevious year\b', updated_line, flags=re.IGNORECASE):
            print(f'Found "previous year" in {file_path}: {line.strip()}')
        if re.search(r'\bnext year\b', updated_line, flags=re.IGNORECASE):
            print(f'Found "next year" in {file_path}: {line.strip()}')
        if re.search(r'\blast year\b|\bprevious year\b', updated_line, flags=re.IGNORECASE):
            print(f'Found "last year or previous year" in {file_path}: {line.strip()}')

        
        updated_line = re.sub(r'\bthis year\b', str(year), updated_line, flags=re.IGNORECASE)
        updated_line = re.sub(r'\bnext year\b', str(year + 1), updated_line, flags=re.IGNORECASE)
        updated_line = re.sub(r'\blast year\b|\bprevious year\b', str(year - 1), updated_line, flags=re.IGNORECASE)

      
        updated_line = remove_corporate_suffixes(updated_line)

        updated_content.append(updated_line)

    with open(file_path, 'w') as file:
        file.writelines(updated_content)
    print(f'Updated {file_path} with company name and year references.')

def remove_corporate_suffixes(text):
   
    return re.sub(r'\b(AG|AG’s|GmbH)\b', '', text, flags=re.IGNORECASE)

def get_company_name_and_year_from_filename(filename):
    
    base_name = os.path.basename(filename)
    match = re.match(r'([A-Za-z]+)_.*_(\d{4})\.txt$', base_name)
    if match:
        company_name = match.group(1)
        
        company_name = re.sub(r'([a-z])([A-Z])', r'\1 \2', company_name)
        year = int(match.group(2))

        print(f'Filename: {filename} -> Company Name: {company_name}, Year: {year}')  
        return company_name, year
    print(f'Filename: {filename} -> Unable to extract company name and year')  
    return None, None

def process_directory(directory):
    # Traverse the directory and process each text file
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                company_name, year = get_company_name_and_year_from_filename(file)
                if company_name and year:
                    replace_text(file_path, company_name, year)
